<a href="https://colab.research.google.com/github/purang2/BERT/blob/main/KERC_1022_PRIVATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **configurations**

### Memory cleaner

In [9]:
memory_clean = True 
#memory_clean = False 

import gc
import torch

if memory_clean: 
  gc.collect()
  torch.cuda.empty_cache()


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
num_kfold = 5 #
NUM_EPOCH = 2
TRAIN_BATCH = 8
EVAL_BATCH = 20
#num_kfold = 7 #
test_private = True

#submission_csv length
if test_private:
  test_num = 4950
else:
  test_num = 2566

#서로 다른 BERT 앙상블 5개 할 것 -> 모델 history는 주석처리를 하여 남겨둘 것 

model_name = "klue/roberta-large" #사용할 모델 1
#model_name = "klue/bert-base" #사용할 모델 1
#model_name = "snunlp/KR-ELECTRA-discriminator" #사용할 모델 2
#model_name = "kykim/bert-kor-base" #사용할 모델 3: colab pro 기준 2epoch: 2~3분 x 5폴드
#model_name = "beomi/KcELECTRA-base" #사용할 모델 4
#model_name = "lighthouse/mdeberta-v3-base-kor-further" #사용할 모델 5


DATE = '10-21'

#submission_comment = f"{DATE} using KLUE-BERT" #최종 저장되는 submission.csv의 이름이 됨
submission_comment = f"{DATE} using KLUE-ROBERTA-L" #최종 저장되는 submission.csv의 이름이 됨
#submission_comment = f"{DATE} using KRELECTRA" #최종 저장되는 submission.csv의 이름이 됨
#submission_comment = f"{DATE} using BERT-Kor-base" #최종 저장되는 submission.csv의 이름이 됨
#submission_comment = f"{DATE} using KcELECTRA-base" #최종 저장되는 submission.csv의 이름이 됨
#submission_comment = f"{DATE} using mDeBERTa-kor" #최종 저장되는 submission.csv의 이름이 됨

#model_nickname = "KLUE-BERT"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨
model_nickname = "KLUE-ROBERTA-L"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨
#model_nickname = "KR-ELECTRA"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨
#model_nickname = "BERT-Kor-base"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨
#model_nickname = "kc-electra-base"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨
#model_nickname = "mDeberta-kor"  #마찬가지로 submission.csv의 이름 자동 지정에 사용됨


## install

In [12]:
!pip install transformers
!pip install datasets #from huggingface
!pip install evaluate
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from datasets import load_dataset
import pandas as pd 

data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"


train_data = pd.read_csv(data_path+"/train_data.tsv", sep='\t')
train_labels = pd.read_csv(data_path+"/train_labels.csv")

train_data

,sentence_id,person,sentence,scene,context
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,NaN
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는"
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
3,4,순경,근무중인데 어딜가?,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
...,...,...,...,...,...
7334,12285,청난,종남이 맛있는것도 사주고 잘 챙겨줘.,S3576,NaN
7335,12286,건강,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,S3576,NaN
7336,12287,종남,아빠. 우리 저거 타.,S3576,NaN
7337,12288,건강,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",S3576,NaN


In [14]:
train_drop = train_data[['person', 'sentence', 'context']]
train_drop['label'] = train_labels['label']
train_pair = train_drop

import numpy as np 

train_pair['context2'] = train_pair['context'].fillna(' ')
train_pair['sentence2'] = train_pair['sentence'].fillna(' ')

train_pair


train_pair = train_pair[['person', 'context2', 'sentence2', 'label']]
train_pair['scene'] = train_data['scene']
train_pair

train = list()
scene_df = train_pair['scene']
scene_df = scene_df.drop_duplicates().reset_index(drop=True)


def mk_scenario(i, df):
  scenario = df.iloc[0]['context2']+" <sep> " 
  if(i==0):
    scenario = scenario + df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario
  elif(i==1):
    scenario = scenario +"\""+ df.iloc[i-1]['sentence2']+"\"" + "라고 " +df.iloc[i-1]['person']+"가 말하자, "+df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario
  elif(i==2):
    scenario = scenario + df.iloc[i-2]['person'] + "가 " + "\""+ df.iloc[i-2]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i-1]['person'] + "가 " + df.iloc[i-1]['sentence2']+"라고 말하자, "+ df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario
  elif(2<i and i <= len(df)-1):
    scenario = scenario + df.iloc[i-3]['person'] + "가 " + "\""+ df.iloc[i-3]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i-2]['person'] + "가 " + df.iloc[i-2]['sentence2']+"라고 말하자, "+ df.iloc[i-1]['person'] + "가 " + "\""+ df.iloc[i-1]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario


def trans_df(train_pair, train):
  for i in scene_df[:]:
    temp = train_pair[train_pair['scene']==i].reset_index(drop=True)
    train+=[mk_scenario(j, temp) for j in range(len(temp))]
    
trans_df(train_pair, train)
train = pd.DataFrame(train, columns=('scenario',))
print(train.iloc[5]['scenario'])

train['label'] = train_pair['label']
train
train_pairs = train

labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }

train_pairs['label'] = [labels[label] for label in train_pairs['label']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대 <sep> 지구대가 "김경사님, 아직 안가셨어요? 시간 다됐을텐데."라고 말하고, 순경가 근무중인데 어딜가?라고 말하자, 지구대가 "다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요."라고 말하고, 순경가 "그럴거 없어." 라고 말하였다.


In [15]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from datasets import Dataset
import pandas as pd 
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import evaluate


max_input_length = 200
#tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")
tokenizer = AutoTokenizer.from_pretrained(model_name)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    metric = evaluate.load("f1")
    #metric = evaluate.load('glue', 'qnli')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    #return metric.compute(predictions=predictions, references=labels)
    return metric.compute(predictions=predictions, references=labels, average='micro')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-discriminator", num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
#klue/roberta-large

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/config.json
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-large",
  "architectures": [
    

In [ ]:

#Gather your training arguments~
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    do_train=True,
    do_eval=True,
#    per_device_train_batch_size=32,
#    per_device_eval_batch_size=32,
    per_device_train_batch_size=TRAIN_BATCH, #16
    per_device_eval_batch_size=EVAL_BATCH, #16
    num_train_epochs=NUM_EPOCH,
#    warmup_steps=1000,
    weight_decay=0.01,
)

trainer = Trainer(
        model=model,
        args=training_args,
        #train_dataset = train_dataset,
        #eval_dataset= val_dataset,
        data_collator=data_collator, 
        tokenizer=tokenizer,  
        compute_metrics = compute_metrics  
    )

def preprocess_function(examples):
    inputs = [doc for doc in examples["scenario"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    return model_inputs


def mk_scenario(i, df):
  scenario = df.iloc[0]['context2']+" <sep> " 
  if(i==0):
    scenario = scenario + df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario
  elif(i==1):
    scenario = scenario +"\""+ df.iloc[i-1]['sentence2']+"\"" + "라고 " +df.iloc[i-1]['person']+"가 말하자, "+df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario
  elif(i==2):
    scenario = scenario + df.iloc[i-2]['person'] + "가 " + "\""+ df.iloc[i-2]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i-1]['person'] + "가 " + df.iloc[i-1]['sentence2']+"라고 말하자, "+ df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario
  elif(2<i and i <= len(df)-1):
    scenario = scenario + df.iloc[i-3]['person'] + "가 " + "\""+ df.iloc[i-3]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i-2]['person'] + "가 " + df.iloc[i-2]['sentence2']+"라고 말하자, "+ df.iloc[i-1]['person'] + "가 " + "\""+ df.iloc[i-1]['sentence2']+"\""+ "라고 말하고, "+ df.iloc[i]['person'] + "가 " +"\""+df.iloc[i]['sentence2'] +"\""+" 라고 말하였다."
    return scenario


def trans_df2(test_pair, test):
  for i in scene_df2[:]:
    temp = test_pair[test_pair['scene']==i].reset_index(drop=True)
    test+=[mk_scenario(j, temp) for j in range(len(temp))]



k_fold_logits = []

#num_kfold = 10
kfold = StratifiedKFold(n_splits=num_kfold, random_state=1104, shuffle=True)

for train_df_idx, val_df_idx in kfold.split(train_pairs, train_pairs['label']):

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

    trainer = Trainer(
            model=model,
            args=training_args,
            #train_dataset = train_dataset,
            #eval_dataset= val_dataset,
            data_collator=data_collator, 
            tokenizer=tokenizer,  
            compute_metrics = compute_metrics,
        )
    

    train_df = train_pairs.iloc[train_df_idx]
    val_df = train_pairs.iloc[val_df_idx]

  
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)

    train_dataset =train_dataset.map(preprocess_function, batched=True).remove_columns(["__index_level_0__",'scenario'])
    val_dataset = val_dataset.map(preprocess_function, batched=True).remove_columns(["__index_level_0__",'scenario'])

    #Collect your model, training args, dataset, data collator, tokenizer in Trainer
    trainer.train_dataset = train_dataset
    trainer.eval_dataset = val_dataset
    trainer.train()



    data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"

    test_data = pd.read_csv(data_path+"/public_test_data.tsv", sep='\t')
    test_pair = test_data[['sentence_id','context','sentence','person', 'scene']]
    
    #PRIVATE
    if test_private:
      test_data_private = pd.read_csv(data_path+"/private_test_data.tsv", sep='\t')
      test_pair_private = test_data_private[['sentence_id','context','sentence','person', 'scene']]
      test_pair = pd.concat([test_pair,test_pair_private],ignore_index=True)
    test_pair['context2'] = test_pair['context'].fillna(' ')
    test_pair['sentence2'] = test_pair['sentence'].fillna(' ')
    #모든 글자 합치기
    test = list()
    scene_df2 = test_pair['scene']
    scene_df2 = scene_df2.drop_duplicates().reset_index(drop=True)
        
    trans_df2(test_pair, test)
    test = pd.DataFrame(test, columns=('scenario',))
    #print("--------------------\n", test)

    #test_pairs = test_pair[['label','new']]
    test['sentence_id'] = test_pair['sentence_id']

    test_dataset = Dataset.from_pandas(test)
    test_dataset =test_dataset.map(preprocess_function, batched=True).remove_columns(['scenario'])


    predictions = trainer.predict(test_dataset)

    k_fold_logits.append(predictions.predictions)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/config.json
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5871
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1468
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,0.719500,0.451681,0.816757
2,0.471600,0.437402,0.861035


...

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5871
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1468


Epoch,Training Loss,Validation Loss,F1
1,0.931200,0.900699,0.616485


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1468
  Batch size = 28


In [ ]:

#predict= k_fold_logits[0] + k_fold_logits[1] + k_fold_logits[2] +k_fold_logits[3] +k_fold_logits[4]
#predict= k_fold_logits[0] + k_fold_logits[1] + k_fold_logits[2] +k_fold_logits[3] +k_fold_logits[4]



for i in range(num_kfold):
  if i==0:
    predict = k_fold_logits[i]
  else:
    predict += k_fold_logits[i]


#predict = k_fold_logits[0]

list1 = []
list2 = []


for i in range(test_num):
  list1.append(i)
  list2.append(-1)



raw_data = {'Id': list1,
            'Predicted': list2
            }
df = pd.DataFrame(raw_data)



labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }

df['Predicted'] = np.argmax(predict, axis=-1)

d = ['euphoria','neutral','dysphoria']

df['Predicted'] = [d[label] for label in df['Predicted']]


df['Id'] = test_dataset['sentence_id']

#x = [d[labels[label]] for label in ['euphoria','euphoria','dysphoria','neutral']]

df



df.to_csv(f'drive/MyDrive/호반우NLP_한국인감정/submission_{model_nickname}_{num_kfold}FOLD_{submission_comment}_batch_{TRAIN_BATCH}_{EVAL_BATCH}.csv', index=False)

df

In [ ]:
df['Predicted'].value_counts()


### Lastly, save predict(np) file for the Ensemble!

In [ ]:
import numpy as np 

ensemble_save =False

if ensemble_save == True:
  np.save(f'drive/MyDrive/호반우NLP_한국인감정/predict_{model_nickname}', predict) #predict_{model_nickname}.npy

## If you wanna get a ensemble results csv, make run_ensemble =  'True'

In [ ]:
#run_ensemble = True
run_ensemble = False

submission_comment_ensemble = '10-21 ensemble'
#run_ensemble = False


In [ ]:
if run_ensemble:
  predict_ensemble = np.load(f'drive/MyDrive/호반우NLP_한국인감정/predict_mDeberta-kor.npy') *0.83
  predict_ensemble += np.load(f'drive/MyDrive/호반우NLP_한국인감정/predict_kc-electra-base.npy') *0.83
  predict_ensemble += np.load(f'drive/MyDrive/호반우NLP_한국인감정/predict_BERT-Kor-base.npy') *0.80
  #predict_ensemble += np.load(f'drive/MyDrive/호반우NLP_한국인감정/predict_KR-ELECTRA.npy') *0.80
  predict_ensemble += np.load(f'drive/MyDrive/호반우NLP_한국인감정/predict_KLUE-ROBERTA-L.npy') *0.85

  raw_data = {'Id': list1,
              'Predicted': list2
              }
  df = pd.DataFrame(raw_data)

  labels = {'euphoria':0,
            'neutral':1,
            'dysphoria':2
            }

  df['Predicted'] = np.argmax(predict_ensemble, axis=-1)

  d = ['euphoria','neutral','dysphoria']

  df['Predicted'] = [d[label] for label in df['Predicted']]


  df['Id'] = test_dataset['sentence_id']

  #x = [d[labels[label]] for label in ['euphoria','euphoria','dysphoria','neutral']]

  df.to_csv(f'drive/MyDrive/호반우NLP_한국인감정/submission_ensemble_{submission_comment_ensemble}.csv', index=False)
  df['Predicted'].value_counts()
